In [ ]:
# ====================================================
# -----------Section 1: Importing Libraries-----------
# ====================================================
# 1.1. Data Manipulation, Statistics, and Feature Engineering
# ====================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
from scipy.stats import randint
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
import math

# ====================================================
# 1.2. Data Visualization
# ====================================================
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import ptitprince as pt



# ============================
# 1.3. Data Splitting, Model Building, and Hyperparameter Tuning
# ============================
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB                   # Naive Bayes classifier
from sklearn.linear_model import LogisticRegression          # Logistic regression
from sklearn.ensemble import RandomForestClassifier # Ensemble methods
from sklearn.svm import SVC                                   # Support vector classifier
from xgboost import XGBClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score         # Cross-validation techniques
from scipy.stats import uniform


# ============================
# 1.4. Model Evaluation and Interpretation
# ============================
import shap                          # SHAP (Shapley values) for model interpretability
import lime                          # LIME (Local Interpretable Model-agnostic Explanations) for model interpretability
import lime.lime_tabular            # LIME (Local Interpretable Model-agnostic Explanations) for model interpretability
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
# ========================================================================
# ----------Section 2: Data Loading, Exploration------------
# ========================================================================
# ----------------------------
# 2.1 Load the dataset
# ----------------------------

esd_df = pd.read_csv(r"https://github.com/EvanQLi/undergrad-diabetes-prediction-xai/blob/main/Early_Stage_Diabetes_Risk_Dataset.csv")
# ----------------------------
# 2.2 Initial dataset inspection
# ----------------------------
#2.2.1 DataFrame Check
print("Dataset row Indices:")
print(esd_df.index)
print("Dataset Column Names:")
print(esd_df.columns)

In [ ]:
#2.2.2 Data Overview
esd_df.head()  # Display the first few rows

In [ ]:
print("\nDataset Info:")
print(esd_df.info())  # Display dataset info

In [ ]:
#2.2.3 Descriptive Statistics
esd_df.describe().T

In [ ]:
# ========================================================================
# ----------Section 3: Data Cleaning------------
# ========================================================================
# ----------------------------
# 3.1 Missing Value Imputation
# ----------------------------
# 3.1.1 Missing Value Check
print("\nMissing Values:")
print(esd_df.isnull().sum())

In [ ]:
# ----------------------------
# 3.2 Duplicate Value Check
# ----------------------------
# 3.2.1 Duplicate Value Check
print(f"Number of duplicate rows: {esd_df.duplicated().sum()}")

In [ ]:
# 3.2.2 Drop Duplicate Rows
esd_df.drop_duplicates(inplace=True)

print("\nDataset Info:")
print(esd_df.info()) 

In [ ]:
# ----------------------------
# 3.3 Outlier Detection using IQR
# ----------------------------
# Define the function to find outliers using IQR
def find_outliers_IQR(data):
	Q1 = data.quantile(0.25)
	Q3 = data.quantile(0.75)
	IQR = Q3 - Q1
	outliers = (data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))
	low_outliers = data < (Q1 - 1.5 * IQR)
	high_outliers = data > (Q3 + 1.5 * IQR)
	return outliers, low_outliers, high_outliers

# 3.3.1 Outlier Detection
# Outliers for 'Age' in esd_df
print("\n Outliers in 'Age'")
alloutliers, lowoutliers, highoutliers = find_outliers_IQR(esd_df['Age'])
print("number of outliers: " + str(alloutliers.sum()))
print("max outlier value: " + str(esd_df['Age'][alloutliers].max()))
print("min outlier value: " + str(esd_df['Age'][alloutliers].min()))
print("number of low outliers: " + str(lowoutliers.sum()))
print("number of high outliers: " + str(highoutliers.sum()))
alloutliers


In [ ]:
# ========================================================================
# ----------Section 4: Exploratory Data Analysis------------
# ========================================================================
# ----------------------------
# 4.1 Target Variable Analysis
# ----------------------------
#4.1.1 Brief overview of target variable
print("\nDistribution of target variable:")
print(esd_df['class'].value_counts())

# 4.1.2 Visualizing the Target Variable (Bar Chart with Percentages)
# Compute class distribution
class_counts = esd_df['class'].value_counts().sort_index()  # Ensure order: 0 (Negative), 1 (Positive)
class_percentage = (class_counts / class_counts.sum()) * 100  # Convert to percentage

# Create bar chart
plt.figure(figsize=(8, 5))
sns.barplot(
    x=class_counts.index,
    y=class_percentage.values,
    palette=['#4682B4', '#FF6347']  # Blue for Negative, Red for Positive
)

# Add percentage labels on bars
for index, value in enumerate(class_percentage):
    plt.text(index, value + 1, f'{value:.1f}%', ha='center', fontsize=12)

# Customize chart aesthetics
plt.title('Percentage of Diabetic vs. Non-Diabetic Cases', fontsize=14)
plt.xlabel('Outcome', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.xticks([0, 1], ['Non-Diabetic', 'Diabetic'])  # Rename x-axis labels
plt.ylim(0, 100)  # Set y-axis limits from 0 to 100%
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()


In [ ]:
# ----------------------------
# 4.2 Age Feature Analysis
# ----------------------------
#4.2.1 Age Distribution Analysis
# Create age groups based on WHO age classification
bins = [0, 44, 60, 75, 90, 100]  # Define age bins
labels = ['Young Age', 'Middle Age', 'Elderly Age', 'Senile Age', 'Long-Livers']  # Define labels for bins
esd_df['age_group'] = pd.cut(esd_df['Age'], bins=bins, labels=labels, right=False)

# Verify the distribution of age groups
print("\nAge Group Distribution:")
print(esd_df['age_group'].value_counts())


In [ ]:
#4.2.2 Age Distribution Across Different Age Groups
# Raincloud plot
import numpy
# Patching the asscalar function to avoid error
def patch_asscalar(a):
    return a.item()

setattr(numpy, "asscalar", patch_asscalar)



# Set figure size
plt.figure(figsize=(12, 6))

diabetic_color = "#FF4500"  # Bright orange-red
non_diabetic_color = "#1E90FF"  # Bright deep blue

# Create Raincloud plot
plt.figure(figsize=(14, 8))

pt.half_violinplot(
    data=esd_df,
    x="Age",
    y="age_group",
    inner=None,
    linewidth=1.5,
    split=True,
    palette="Set3",
    edgecolor='white',
    dodge=True,
    orient="h",
)

# Add boxplot
sns.boxplot(
    data=esd_df,
    x="Age",
    y="age_group",
    width=0.15,
    zorder=10,
    boxprops = {'facecolor':'none', "zorder":10},
    whiskerprops = {'linewidth':2, "zorder":10}
    )   

# Add swarmplot
sns.swarmplot(
    data=esd_df,
    x="Age",
    y="age_group",
    hue="class",
    size=4,
    edgecolor='white',
    linewidth=0.5,
    palette=[diabetic_color, non_diabetic_color],
    zorder=0
)

# Add Count Labels to Each Age Group
age_group_counts = esd_df.groupby(["age_group", "class"]).size().unstack(fill_value=0)
# ✅ Get rightmost x-limit for alignment
x_max = plt.xlim()[1]  
text_x = x_max + 5  # Move labels further right

# ✅ Add Count Labels to Each Age Group
for i, (age_group, row) in enumerate(age_group_counts.iterrows()):
    non_diabetic_count = row.get(0, 0)  # Get count safely
    diabetic_count = row.get(1, 0)  # Get count safely

    plt.text(text_x, i + 0.15, f"Non-Diabetic: {non_diabetic_count}", fontsize=13,
             fontweight="bold", color=non_diabetic_color, ha="left")

    plt.text(text_x, i - 0.15, f"Diabetic: {diabetic_count}", fontsize=13,
             fontweight="bold", color=diabetic_color, ha="left")

# Improve plot readability
plt.title("Age Distribution Across Different Age Groups", fontsize=16, fontweight="bold")
plt.xlabel("Age", fontsize=14)
plt.ylabel("Age Group", fontsize=14)
plt.xticks(rotation=45, fontsize=13)
plt.yticks(fontsize=13)
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.legend(title="Diabetic Status", title_fontsize="13", fontsize="12", loc="upper right")

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#4.2.3 Drop the 'age_group' column
esd_df.drop(columns=['age_group'], inplace=True)

esd_df_copy = esd_df.copy()

In [ ]:
# ----------------------------
# 4.3 Gender Feature Analysis
# ----------------------------
#4.3.1 Overview of Gender Distribution
gender_counts = esd_df['Gender'].value_counts().sort_index()  # Ensure order: 1(Male), 0 (Female)
gender_percentage = (gender_counts / gender_counts.sum()) * 100

# Print summary
print("\nGender Distribution:")
print(gender_counts)
print("\nGender Percentage Distribution:")
print(gender_percentage.round(2))

In [ ]:
#4.3.2 Gender Distribution Bar Chart 
plt.figure(figsize=(8, 5))
sns.barplot(
    x=gender_counts.index,
    y=gender_percentage.values,
    palette=['#ff7f0e', '#1f77b4']  # Blue for Male, Orange for Female
)

# Add percentage labels on bars
for index, value in enumerate(gender_percentage):
    plt.text(index, value + 1, f'{value:.1f}%', ha='center', fontsize=12, fontweight='bold')

# Customize chart aesthetics
plt.title('Gender Distribution Percentage in ESD', fontsize=14, fontweight='bold')
plt.xlabel('Gender', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.xticks([0, 1], ['Female', 'Male'])  # Ensure labels match encoded values
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

# 3. Gender Distribution Across Diabetes Classes
plt.figure(figsize=(8, 5))
ax = sns.countplot(
    x='Gender', hue='class', data=esd_df, 
    palette=['#FF6347', '#4682B4'],  # Blue for Negative, Red for Positive
    edgecolor='black'
)

# Add count labels on each bar
for p in ax.patches:
    ax.annotate(
        f'{int(p.get_height())}',  # Convert float to integer for better readability
        (p.get_x() + p.get_width() / 2., p.get_height()),  # Position label at the top of the bar
        ha='center', va='bottom', fontsize=12, fontweight='bold', color='black'
    )

# Customize chart aesthetics
plt.title('Gender Distribution by Diabetes Status', fontsize=14, fontweight='bold')
plt.xlabel('Gender', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks([0, 1], ['Male', 'Female'])  # Ensure labels match dataset values
plt.legend(title='Diabetes Status', labels=['Positive', 'Negative'])
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

In [ ]:
# ----------------------------
# 4.4 Binary Features Analysis
# ----------------------------
#4.4.1 Data Encoding for Binary Features Analysis

# Convert class labels to binary
esd_df_copy['class'] = esd_df_copy['class'].map({'Positive': 1, 'Negative': 0})

# Convert Categorical Features to Numerical via Label Encoding

# Create LabelEncoder instance
le = LabelEncoder()

# Apply to all binary categorical columns
binary_cols = ['Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss','weakness',
               'Polyphagia','Genital thrush','visual blurring','Itching','Irritability',
               'delayed healing','partial paresis','muscle stiffness','Alopecia',
               'Obesity'
               ]  
for col in binary_cols:
    esd_df_copy[col] = le.fit_transform(esd_df[col])

# Check the transformed dataset
print(esd_df_copy.head())





In [ ]:
#------------------------
# #4.4.2 Compute Distribution Rate of Binary Features
#------------------------
# Extract binary features (excluding target variable)
binary_features = [col for col in esd_df_copy.columns if esd_df_copy[col].nunique() == 2 and col != 'class'and col != 'Gender']

# Compute proportion of "Yes" (1) and "No" (0) responses
binary_summary = pd.DataFrame({
    'Feature': binary_features,
    'Yes (%)': [(esd_df_copy[col].mean()) * 100 for col in binary_features],  # Mean gives proportion of 1s
    'No (%)': [(1 - esd_df_copy[col].mean()) * 100 for col in binary_features]
})

# Rank features by the highest "Yes" rate
binary_summary = binary_summary.sort_values(by='Yes (%)', ascending=False)

# Print ranked summary
print("\nBinary Feature Distribution Ranked by 'Yes' Rate:")
print(binary_summary.round(2))

In [ ]:
#------------------------
# #4.4.3 Visualize Distribution Rate of Binary Features
#------------------------
# Convert data to long format for visualization
binary_summary_melted = binary_summary.melt(id_vars=['Feature'], var_name='Response', value_name='Percentage')
# Set figure size
plt.figure(figsize=(16, 10))

# Create stacked horizontal bar chart
binary_summary.set_index("Feature")[["Yes (%)", "No (%)"]].plot(
    kind="barh", stacked=True, color=['#b2d7ca', '#3884a5'], edgecolor="black", figsize=(16, 10),fontsize=12
)

# Improve readability
plt.xlabel("Percentage (%)", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.title("Distribution Rate of Binary Features", fontsize=14, fontweight="bold")


# Add percentage labels inside bars
for i, (yes, no) in enumerate(zip(binary_summary["Yes (%)"], binary_summary["No (%)"])):
    plt.text(yes / 2, i - 0.15, f"{yes:.1f}%", fontsize=12, color="black", ha="center", fontweight="bold" )  # Yes inside bar
    plt.text(yes + (no / 2), i - 0.15, f"{no:.1f}%", fontsize=12, color="black", ha="center", fontweight="bold" )  # No inside bar

# Adjust grid
plt.grid(axis="x", linestyle="--", alpha=0.7)

# Show plot
plt.tight_layout(rect=[0, 0, 0.85, 1])
# Add legend outside the plot
plt.legend(["Yes (%)", "No (%)"], title="Response", fontsize=12, loc="center left", bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
#------------------------
#4.4.4 Visualize Binary Features by Diabetes Status
#------------------------
# Automatically determine grid size based on number of binary features
num_features = len(binary_features)
num_cols = 3  # Fixed number of columns
num_rows = math.ceil(num_features / num_cols)  # Compute required rows

# Set up a flexible grid of subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, num_rows * 3))  
axes = axes.flatten()  # Flatten for easy iteration


# Loop through each binary feature and plot its distribution across diabetes classes
for i, col in enumerate(binary_features):
    ax = sns.countplot(
        x=col, hue='class', data=esd_df_copy, order=[0, 1],  # Ensure order: 1 (Yes), 0 (No)
        palette=['#1E90FF', '#FF4500'], edgecolor='black', ax=axes[i]
    )
    # ✅ Rename X-axis labels from 0 → "No" and 1 → "Yes"
    ax.set_xticklabels(["No", "Yes"])
    # Get the maximum y-axis value for scaling
    max_count = max([p.get_height() for p in ax.patches if p.get_height() > 0])

    # Set y-axis limit slightly above max count to prevent labels from overlapping
    ax.set_ylim(0, max_count * 1.15)  

    # Add count labels dynamically positioned within the plot
    for p in ax.patches:
        if p.get_height() > 0:  # Skip labels for bars with zero height
            ax.annotate(
                f'{int(p.get_height())}',  
                (p.get_x() + p.get_width() / 2., p.get_height() + (max_count * 0.02)),  
                ha='center', fontsize=10, fontweight='bold', color='black'
            )

    # Customize titles and labels
    ax.set_title(f'{col} by Diabetes Status', fontsize=12, fontweight='bold')
    ax.set_xlabel(col, fontsize=13)
    ax.set_ylabel('Count', fontsize=13)

    # Remove individual legends to avoid duplicates
    if i != len(binary_features):  
        ax.get_legend().remove()

# Remove any extra blank subplots that were created
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Add a single legend on the right side of the figure
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, ['Negative', 'Positive'], title='Diabetes Status', loc='center right', fontsize=14)

# Adjust layout for better readability
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Leave space on the right for the legend
plt.show()

In [ ]:
#------------------------
# 4.4.4 Visualize Binary Features by Diabetes Status (Using Probabilities with Percentage Labels)
#------------------------

# Automatically determine grid size based on number of binary features
num_features = len(binary_features)
num_cols = 3  # Fixed number of columns
num_rows = math.ceil(num_features / num_cols)  # Compute required rows

# Set up a flexible grid of subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, num_rows * 3))
axes = axes.flatten()  # Flatten for easy iteration

# Loop through each binary feature and plot its probabilities across diabetes classes
for i, col in enumerate(binary_features):
    # Calculate probabilities for each class
    probabilities = (
        esd_df_copy.groupby(['class', col])
        .size()
        .groupby(level=0)
        .apply(lambda x: x / x.sum())
        .unstack()
    )
    
    # Plot the probabilities
    ax = probabilities.plot(
        kind='bar',
        stacked=True,
        color=['#1E90FF', '#FF4500'],  # Blue for "No", Red for "Yes"
        edgecolor='black',
        ax=axes[i]
    )
    
    # Add percentage labels inside the bars
    for p in ax.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy()
        if height > 0:  # Avoid labeling bars with zero height
            ax.text(
                x + width / 2, y + height / 2,
                f'{height * 100:.1f}%',  # Convert to percentage
                ha='center', va='center', fontsize=10, color='white', fontweight='bold'
            )
    
    # Customize titles and labels
    axes[i].set_title(f'{col} by Diabetes Status (Probabilities)', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Diabetes Status', fontsize=13)
    axes[i].set_ylabel('Probability', fontsize=13)
    axes[i].set_xticklabels(['Negative', 'Positive'], rotation=0)  # Rename x-axis labels

  # Remove individual legends to avoid duplicates
    if i != len(binary_features):  
        ax.get_legend().remove()

# Remove any extra blank subplots that were created
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Add a single legend on the right side of the figure
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, ['Negative', 'Positive'], title='Diabetes Status', loc='lower right', bbox_to_anchor=(0.9,0.1),fontsize=14)

# Adjust layout for better readability
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------
# 4.5 Correlation Analysis
# ----------------------------
#4.5.1 Heatmap of correlations
plt.figure(figsize=(10, 8))
sns.heatmap(esd_df_copy.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Features', fontsize=16)
plt.show()

In [ ]:
# 3.2 Top correlated features with Class
correlations = esd_df_copy.corr()['class'].drop('class').sort_values(ascending=False)
print("Top Correlated Features with Class:")
print(correlations)

# Set figure size
plt.figure(figsize=(12, 8))

# Create bar plot
sns.barplot(x=correlations.values, y=correlations.index, dodge=False, palette='Set2')

# Add count labels on each bar
for i, v in enumerate(correlations):
    plt.text(v, i, f"{v:.2f}", color='black', va='center', fontsize=12)

# Improve readability
plt.xlabel("Correlation Coefficient", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.title('Feature Correlations with Class', fontsize=16, fontweight="bold")
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# ========================================================================
# ----------Section 5: Feature Engineering------------
# ========================================================================
# ----------------------------
# 5.1 Data Encoding
# ----------------------------
# Convert class labels to binary
esd_df['class'] = esd_df['class'].map({'Positive': 1, 'Negative': 0})

# Convert Categorical Features to Numerical via Label Encoding

# Create LabelEncoder instance
le = LabelEncoder()

# Apply to all binary categorical columns
binary_cols = ['Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss','weakness',
               'Polyphagia','Genital thrush','visual blurring','Itching','Irritability',
               'delayed healing','partial paresis','muscle stiffness','Alopecia',
               'Obesity'
               ]  
for col in binary_cols:
    esd_df[col] = le.fit_transform(esd_df[col])

# Check the transformed dataset
print(esd_df.head())


In [ ]:
# ----------------------------
# 5.2 Data Splitting
# ----------------------------

#5.2.1 Split the dataset into features (X) and target variable (y)
X = esd_df.drop(columns=['class'])
y = esd_df['class']

# Display the shapes of X and y to verify the split
print("Features (X) shape:", X.shape)
print("Target (y) shape:", y.shape)

In [ ]:
#5.2.2 Split the dataset into training and testing sets(7:3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Validate class distribution in train and test splits
print("Training class distribution:\n", y_train.value_counts(normalize=True))
print("Testing class distribution:\n", y_test.value_counts(normalize=True))

In [ ]:
# ----------------------------
# 5.3 Data SMOTE
# ----------------------------
#not to do, although the dataset is imbalanced, 
#keep the original dataset could be better for the model.

In [ ]:
# ----------------------------
# 5.4 Data Scaling
# ----------------------------
# Create StandardScaler instance
scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Transform the training and testing data for models not tree-based 
X_train_scaled[['Age']]= scaler.fit_transform(X_train_scaled[['Age']])
X_test_scaled[['Age']] = scaler.transform(X_test_scaled[['Age']])

# Display the shape of the splits
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# ========================================================================
# ----------Section 6: Model Building and Hyperparameter Tuning-----------
# ========================================================================
# ----------------------------
# 6.1 Naive Bayes
# ----------------------------
# 6.1.1 Model Building and Hyperparameter Tuning using GridSearchCV

# Initialize the Naive Bayes classifier
nb = GaussianNB()

# Initialize the RandomizedSearchCV object
nb_search = GridSearchCV(
    estimator=nb, 
    param_grid={"var_smoothing": np.logspace(0,-2, num=100)},  # Use GridSearchCV 
    cv=10,  # 10-fold cross-validation
    scoring="roc_auc",  # Optimize for ROC-AUC
    n_jobs=-1,  # Use all available processors
    verbose=1  # Display progress
)

# Fit the GridSearchCV object to the training data
nb_search.fit(X_train_scaled, y_train)

# Get the best parameters
nb_best_params = nb_search.best_params_
print("Best Parameters for Naive Bayes:", nb_best_params)

In [ ]:
#6.1.2 Model Evaluation

# Initialize the Naive Bayes classifier with the best parameters
nb_best_model = GaussianNB(var_smoothing=nb_best_params['var_smoothing'])
nb_best_model.fit(X_train_scaled, y_train)

# Predict the target variable
y_pred_nb = nb_best_model.predict(X_test_scaled)


# Calculate evaluation metrics
nb_acc = accuracy_score(y_test, y_pred_nb)
nb_conf_matrix = confusion_matrix(y_test, y_pred_nb)
nb_roc_auc = roc_auc_score(y_test, y_pred_nb)
nb_f1 = f1_score(y_test, y_pred_nb)
nb_precision = precision_score(y_test, y_pred_nb)
nb_recall = recall_score(y_test, y_pred_nb)

# Print evaluation results
print(f"Naive Bayes Accuracy: {nb_acc:.2f}")
print("Naive Bayes Confusion Matrix:")
print(nb_conf_matrix)
print("\nClassification Report:\n", classification_report(y_test, y_pred_nb))
print(f"ROC-AUC Score: {nb_roc_auc:.2f}")
print(f"F1-Score: {nb_f1:.2f}")
print(f"Precision: {nb_precision:.2f}")
print(f"Recall: {nb_recall:.2f}")

# Visualize the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(nb_conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Diabetes", "Diabetes"], yticklabels=["No Diabetes", "Diabetes"])
plt.title("Naive Bayes Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
# ----------------------------
# 6.2 Logistic Regression
# ----------------------------
# 6.2.1 Model Building and Hyperparameter Tuning using GridSearchCV
# Initialize the Logistic Regression classifier
lr = LogisticRegression()

# Define the hyperparameter grid for tuning
lr_params = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],  # Regularization strengths
    "penalty": ["l2", "l1"],  # Regularization penalties
    "solver": ["liblinear", "saga"],  # Solvers compatible with both l1 and l2 penalties
}

# Initialize GridSearchCV
lr_grid = GridSearchCV(
    estimator=lr,
    param_grid=lr_params,
    scoring="roc_auc",  # Optimize for ROC-AUC
    cv=10,  # 10-fold cross-validation
    verbose=1,  # Display progress during fitting
    n_jobs=-1  # Use all available processors
)

# Fit the model with hyperparameter tuning
lr_grid.fit(X_train_scaled, y_train)

# Retrieve the best hyperparameters
lr_best_params = lr_grid.best_params_
print("Best Parameters for Logistic Regression:", lr_best_params)

In [ ]:
#6.2.2 Model Evaluation

# Initialize the Logistic Regression classifier with the best parameters
lr_best_model = LogisticRegression(**lr_best_params)

# Fit the model on the training data
lr_best_model.fit(X_train_scaled, y_train)

# Predict the target variable
y_pred_lr = lr_best_model.predict(X_test_scaled)

# Calculate evaluation metrics
lr_acc = accuracy_score(y_test, y_pred_lr)
lr_conf_matrix = confusion_matrix(y_test, y_pred_lr)
lr_roc_auc = roc_auc_score(y_test, y_pred_lr)
lr_f1 = f1_score(y_test, y_pred_lr)
lr_precision = precision_score(y_test, y_pred_lr)
lr_recall = recall_score(y_test, y_pred_lr)

# Print evaluation results
print(f"Logistic Regression Accuracy: {lr_acc:.2f}")
print("Logistic Regression Confusion Matrix:")
print(lr_conf_matrix)
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))
print(f"ROC-AUC Score: {lr_roc_auc:.2f}")
print(f"F1-Score: {lr_f1:.2f}")
print(f"Precision: {lr_precision:.2f}")
print(f"Recall: {lr_recall:.2f}")

# Visualize the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(lr_conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Diabetes", "Diabetes"], yticklabels=["No Diabetes", "Diabetes"])
plt.title("Logistic Regression Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# ----------------------------
# 6.3 Random Forest
# ----------------------------

# 6.3.1 Model Building and Hyperparameter Tuning


# Initialize the Random Forest classifier
rf = RandomForestClassifier()  # Set random_state for reproducibility

# Define the hyperparameter grid
rf_params = {
    "n_estimators": [100, 200, 300, 400, 500],  # Number of trees in the forest
    "criterion": ["gini", "entropy"],  # Split criterion
    "max_depth": [10, 20, 30, 40, 50, None],  # Maximum depth of the tree
    "min_samples_split": [2, 5, 10],  # Minimum samples to split an internal node
    "min_samples_leaf": [1, 2, 4],  # Minimum samples required to be at a leaf node
    "max_features": ["sqrt", "log2", None],  # Number of features to consider at each split
}

# Initialize GridSearchCV for hyperparameter tuning
rf_grid = GridSearchCV(
    estimator=rf,
    param_grid=rf_params,
    scoring="roc_auc",  # Optimize for ROC-AUC
    cv=10,  # 10-fold cross-validation
    verbose=1,  # Display progress
    n_jobs=-1  # Use all available processors
)

# Fit the GridSearchCV object to find the best hyperparameters
rf_grid.fit(X_train, y_train)

# Retrieve the best hyperparameters
rf_best_params = rf_grid.best_params_
print("Best Parameters for Random Forest:", rf_best_params)

In [ ]:

# 6.3.2 Model Evaluation

# Initialize the Random Forest classifier with the best hyperparameters
rf_best_model = RandomForestClassifier(**rf_best_params)

# Fit the final model on the training data
rf_best_model.fit(X_train, y_train)


# Predict the outcomes for the test set
rf_pred = rf_best_model.predict(X_test)
rf_pred_prob = rf_best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Calculate evaluation metrics
rf_acc = accuracy_score(y_test, rf_pred)
rf_conf_matrix = confusion_matrix(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_prob)
rf_f1 = f1_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)

# Print evaluation results
print(f"Random Forest Accuracy: {rf_acc:.3f}")
print("Random Forest Confusion Matrix:")
print(rf_conf_matrix)
print("\nClassification Report:\n", classification_report(y_test, rf_pred))
print(f"Random Forest ROC-AUC Score: {rf_roc_auc:.3f}")
print(f"F1-Score: {rf_f1:.3f}")
print(f"Precision: {rf_precision:.3f}")
print(f"Recall: {rf_recall:.3f}")

# Visualize the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(rf_conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Diabetes", "Diabetes"], yticklabels=["No Diabetes", "Diabetes"])
plt.title("Random Forest Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# ----------------------------
# 6.4 Support Vector Machine (SVM)
# ----------------------------

# 6.4.1 Model Building and Hyperparameter Tuning

# Initialize the Support Vector Machine classifier

# svm = SVC()  # Set random_state for reproducibility

# Define the hyperparameter grid
# svm_params = {
#     "C": [1, 10],  # Regularization parameter
#     "kernel": ["linear", "rbf"],  # Kernel type
#     "gamma": ["scale", "auto"],  # Kernel coefficient
# }   
#     # Running to slow with parameters below(need to be commented out),
#     #"C": [0.1, 1, 10, 100],  # Wider range of regularization strengths
#     # "kernel": ["linear", "rbf", "poly"],  # Adding polynomial kernel for flexibility
#     # "degree": [2, 3],  # Polynomial kernel degree (only applicable for "poly")
#     # "gamma": ["scale", "auto"]  # Kernel coefficient

# Initialize GridSearchCV for hyperparameter tuning
# svm_grid = GridSearchCV(
#     estimator=svm,
#     param_grid=svm_params,
#     scoring="roc_auc",  # Optimize for ROC-AUC
#     cv=10,  # 10-fold cross-validation
#     verbose=1,  # Display progress
#     n_jobs=-1  # Use all available processors
# )

# Fit the GridSearchCV object to find the best hyperparameters
# svm_grid.fit(X_train_scaled, y_train)

# Retrieve the best hyperparameters
# svm_best_params = svm_grid.best_params_
# print("Best Parameters for Support Vector Machine:", svm_best_params)

In [ ]:
# 6.4.2 Model Evaluation

# Initialize the SVM classifier with the best hyperparameters
# svm_best_model = SVC(**svm_best_params)

# Fit the final model on the training data
# svm_best_model.fit(X_train_scaled, y_train)

# Predict the outcomes for the test set
# svm_pred = svm_best_model.predict(X_test_scaled)

# Calculate evaluation metrics
# svm_acc = accuracy_score(y_test, svm_pred)
# svm_conf_matrix = confusion_matrix(y_test, svm_pred)
# svm_roc_auc = roc_auc_score(y_test, svm_pred)
# svm_f1 = f1_score(y_test, svm_pred)
# svm_precision = precision_score(y_test, svm_pred)
# svm_recall = recall_score(y_test, svm_pred)

# Print evaluation results
# print(f"SVM Accuracy: {svm_acc:.3f}")
# print("SVM Confusion Matrix:")
# print(svm_conf_matrix)
# print("\nClassification Report:\n", classification_report(y_test, svm_pred))
# print(f"SVM ROC-AUC Score: {svm_roc_auc:.3f}")
# print(f"F1-Score: {svm_f1:.3f}")
# print(f"Precision: {svm_precision:.3f}")
# print(f"Recall: {svm_recall:.3f}")

# Visualize the confusion matrix
# plt.figure(figsize=(6, 4))
# sns.heatmap(svm_conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Diabetes", "Diabetes"], yticklabels=["No Diabetes", "Diabetes"])
# plt.title("SVM Confusion Matrix")
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# plt.show()


In [ ]:
# ----------------------------
# 6.5 Extreme Gradient Boosting (XGBoost)
# ----------------------------

# 6.5.1 Model Building and Hyperparameter Tuning

# Initialize the Gradient Boosting (XGBoost) classifier
gb = XGBClassifier()

# Define the hyperparameter grid
gb_params = {
    'n_estimators': [50, 100, 150],  # Number of trees
    'learning_rate': [0.01, 0.1, 0.5],  # Step size shrinkage
    'max_depth': [2, 3, 4, 5],  # Maximum depth of trees
    'subsample': [0.5, 0.8, 1.0],  # Fraction of samples for training
    'colsample_bytree': [0.1, 0.2, 0.3, 0.5],  # Fraction of features per tree
    'colsample_bylevel': [0.1, 0.2, 0.3, 0.5],  # Fraction of features per level
    'min_child_weight': [1, 3, 5, 7],  # Minimum sum of weights of child nodes
}

# Initialize GridSearchCV
gb_grid = GridSearchCV(
    estimator=gb,
    param_grid=gb_params,
    scoring="roc_auc",  # Optimize for ROC-AUC
    cv=10,  # 10-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all available processors
)

# Fit the GridSearchCV object to find the best hyperparameters
gb_grid.fit(X_train, y_train)

# Retrieve the best hyperparameters and best estimator
gb_best_params = gb_grid.best_params_
gb_model = gb_grid.best_estimator_
print("Best Parameters for Gradient Boosting (XGBoost):", gb_best_params)

In [ ]:

# 6.5.2 Model Evaluation

# Initialize the Gradient Boosting (XGBoost) classifier with the best hyperparameters
gb_best_model = XGBClassifier(**gb_best_params)

# Fit the final model on the training data
gb_best_model.fit(X_train, y_train)

# Predict the outcomes for the test set
gb_pred = gb_best_model.predict(X_test)
gb_pred_prob = gb_best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate evaluation metrics
gb_acc = accuracy_score(y_test, gb_pred)
gb_conf_matrix = confusion_matrix(y_test, gb_pred)
gb_roc_auc = roc_auc_score(y_test, gb_pred_prob)
gb_f1 = f1_score(y_test, gb_pred)
gb_precision = precision_score(y_test, gb_pred)
gb_recall = recall_score(y_test, gb_pred)

# Calculate sensitivity and specificity
sensitivity = gb_conf_matrix[1, 1] / (gb_conf_matrix[1, 1] + gb_conf_matrix[1, 0])
specificity = gb_conf_matrix[0, 0] / (gb_conf_matrix[0, 0] + gb_conf_matrix[0, 1])

# Print evaluation results
print(f"Gradient Boosting Accuracy: {gb_acc:.2f}")
print("Gradient Boosting Confusion Matrix:")
print(gb_conf_matrix)
print("\nClassification Report:\n", classification_report(y_test, gb_pred))
print(f"ROC-AUC Score: {gb_roc_auc:.2f}")
print(f"F1-Score: {gb_f1:.2f}")
print(f"Precision: {gb_precision:.2f}")
print(f"Recall: {gb_recall:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

# Visualize the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(gb_conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Diabetes", "Diabetes"], yticklabels=["No Diabetes", "Diabetes"])
plt.title("Gradient Boosting Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
# ----------------------------
# 6.6 Stacking Classifier
# ----------------------------
%pip install mlxtend

from mlxtend.classifier import StackingCVClassifier

# Initialize the base classifiers
base_classifiers = [
    LogisticRegression(**lr_best_params),
    RandomForestClassifier(**rf_best_params),
    XGBClassifier(**gb_best_params)
]

# Define the meta-classifier
meta_classifier = RandomForestClassifier()

# Initialize the StackingCV classifier
stacking = StackingCVClassifier(
    classifiers=base_classifiers,
    meta_classifier=meta_classifier,
    cv=10,  # 10-fold cross-validation
    stratify=True,  # Use stratified folds
    shuffle=True,  # Shuffle the data
    n_jobs=-1,  # Use all available processors
)

# Fit the StackingCV classifier
stacking.fit(X_train_scaled, y_train)



In [ ]:
# Predict the target variable
y_stacking_pred = stacking.predict(X_test)

# Calculate evaluation metrics
stacking_acc = accuracy_score(y_test, y_stacking_pred)
stacking_conf_matrix = confusion_matrix(y_test, y_stacking_pred)
stacking_roc_auc = roc_auc_score(y_test, y_stacking_pred)
stacking_f1 = f1_score(y_test, y_stacking_pred)
stacking_precision = precision_score(y_test, y_stacking_pred)
stacking_recall = recall_score(y_test, y_stacking_pred)

# Print evaluation results
print(f"Stacking Classifier Accuracy: {stacking_acc:.2f}")
print("Stacking Classifier Confusion Matrix:")
print(stacking_conf_matrix)
print("\nClassification Report:\n", classification_report(y_test, y_stacking_pred))
print(f"ROC-AUC Score: {stacking_roc_auc:.2f}")
print(f"F1-Score: {stacking_f1:.2f}")
print(f"Precision: {stacking_precision:.2f}")
print(f"Recall: {stacking_recall:.2f}")

# Visualize the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(stacking_conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Diabetes", "Diabetes"], yticklabels=["No Diabetes", "Diabetes"])
plt.title("Stacking Classifier Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()





In [ ]:
# ========================================================================
# ----------Section 7: Model Performance Comparison------------
# ========================================================================

# 7.1 Model Performance Metrics 

# Create a DataFrame to display the model performance metrics

# Initialize data for the model performance DataFrame
data = {
    "Model": ["NB", "LR", "RF", "XGB", "Stacking"],
    "Accuracy": [round(nb_acc, 3), round(lr_acc, 3), round(rf_acc, 3), round(gb_acc, 3), round(stacking_acc, 3)],
    "ROC-AUC": [round(nb_roc_auc, 3), round(lr_roc_auc, 3), round(rf_roc_auc, 3), round(gb_roc_auc, 3), round(stacking_roc_auc, 3)],
    "F1-Score": [round(nb_f1, 3), round(lr_f1, 3), round(rf_f1, 3), round(gb_f1, 3), round(stacking_f1, 3)],
    "Precision": [round(nb_precision, 3), round(lr_precision, 3), round(rf_precision, 3), round(gb_precision, 3), round(stacking_precision, 3)],
    "Recall": [round(nb_recall, 3), round(lr_recall, 3), round(rf_recall, 3), round(gb_recall, 3), round(stacking_recall, 3)]
}

# Create the DataFrame
model_comparison = pd.DataFrame(data)

# Display the DataFrame
print("\nModel Comparison:")
print(model_comparison)


# 7.2 Visualizing Model  Performance


# 7.2.1 Create a Bar Plot for Model Comparison

# Convert the data to long format
model_comparison_long = pd.melt(model_comparison, id_vars=["Model"], var_name="Metric", value_name="Value")

# Set the figure size
plt.figure(figsize=(14, 8))

# customed color
colors = ["#e2edc9", "#a4d3b7", "#65c4b9", "#26a7c8", "#607bbc"]
# Create a bar plot to compare the model performance metrics
sns.barplot(x="Model", y="Value", hue="Metric", data=model_comparison_long, palette=colors)

# Customize the plot
plt.title("Model Performance Comparison", fontsize=16)
plt.xlabel("Model", fontsize=16)
plt.ylabel("Performance Metric", fontsize=16)

plt.legend(fontsize=12, bbox_to_anchor=(1, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# ========================================================================
# ----------Section 8: Feature Interpretability------------
# ========================================================================


# ----------------------------
# 8.1 Importance ranking of features in the model built-in
# ----------------------------
# 8.1.1 Random Forest Feature Importance
# Extract feature importances from the Random Forest model
rf_feature_importance = rf_best_model.feature_importances_

# Create a DataFrame to display the feature importance values
rf_fi_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf_feature_importance
})

# Sort the DataFrame based on feature importance
rf_fi_df = rf_fi_df.sort_values(by="Importance", ascending=False).reset_index(drop=True)

# Display the feature importances
print("\nRandom Forest Feature Importance:")
print(rf_fi_df)



#plot the feature importance
# Plot the feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=rf_fi_df, palette="viridis")

# Add percentage labels on each bar
for index, value in enumerate(rf_fi_df["Importance"]):
    plt.text(value, index, f'{value:.2%}', color='black', va="center", fontsize=6)

plt.title("Random Forest Feature Importance", fontsize=16)
plt.xlabel("Importance", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.show()


In [ ]:
# 8.1.2 Gradient Boosting Feature Importance

# Extract feature importances from the Gradient Boosting model
gb_feature_importance = gb_best_model.feature_importances_

# Create a DataFrame to display the feature importance values
gb_fi_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": gb_feature_importance
})

# Sort the DataFrame based on feature importance
gb_fi_df = gb_fi_df.sort_values(by="Importance", ascending=False).reset_index(drop=True)

# Display the feature importances
print("\nGradient Boosting Feature Importance:")
print(gb_fi_df)


# Plot the feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=gb_fi_df, palette="viridis")

# Add percentage labels on each bar
for index, value in enumerate(gb_fi_df["Importance"]):
    plt.text(value, index, f'{value:.2%}', color='black', va="center", fontsize=6)

plt.title("Gradient Boosting Feature Importance", fontsize=16)
plt.xlabel("Importance", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.show()

In [ ]:
# ----------------------------
# 8.2 Importance ranking of features by SHAP values
# ----------------------------
# ----------------------------
# 8.2.1 SHAP Analysis for Random Forest
# ----------------------------

# Initialize the SHAP explainer for the best Random Forest model
rf_explainer = shap.Explainer(rf_best_model)

# Calculate SHAP values
rf_shap_values = rf_explainer.shap_values(X_test)



#Visualize the SHAP values
shap.summary_plot(rf_shap_values[:,:,1], X_test,  feature_names=X_train.columns)

In [ ]:
#8.2.2 SHAP Analysis for Gradient Boosting
# Create object that can calculate shap values for Gradient Boosting model
explainer_gb = shap.TreeExplainer(gb_best_model)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of X_test rather than a single row, to have more data for plot.
shap_values_gb = explainer_gb.shap_values(X_test)

# Make plot for Gradient Boosting model
shap.summary_plot(shap_values_gb, X_test, feature_names=X.columns)

In [ ]:
# ----------------------------
# 8.3 Local Interpretability using LIME
# ----------------------------
# ----------------------------
# 8.3.1 True Positive Case in ESD
# ----------------------------
# Find the index of a true positive case in random forest, xgboost, and stacking
true_positive_idx = y_test[(y_test == 1) & (rf_pred == 1) & (gb_pred ==1) & (y_stacking_pred == 1)].index[0]

# Extract the features for the true positive case
ESD_X_true_positive = X_test.loc[[true_positive_idx]]

print("\nTrue Positive Case:")
print(ESD_X_true_positive)



In [ ]:

# Lime explainer for the true positive case via random forest
# Initialize the LimeTabularExplainer
explainer = LimeTabularExplainer(X_train.values, mode="classification", feature_names=X.columns, class_names=["Negative", "Positive"])
# Explain the true positive case
exp = explainer.explain_instance(ESD_X_true_positive.values[0], rf_best_model.predict_proba, num_features=16, top_labels=1)

# Display the explanation
exp.show_in_notebook(show_table=True)




In [ ]:
# Lime explainer for the true positive case via XGBoost
# Initialize the LimeTabularExplainer
explainer = LimeTabularExplainer(X_train.values, mode="classification", feature_names=X.columns, class_names=["Negative", "Positive"])
# Explain the true positive case
exp = explainer.explain_instance(ESD_X_true_positive.values[0], gb_best_model.predict_proba, num_features=16, top_labels=1)

# Display the explanation
exp.show_in_notebook(show_table=True)



In [ ]:
# Lime explainer for the true positive case via stacking classifier

# Initialize the LimeTabularExplainer
explainer = LimeTabularExplainer(X_train.values, mode="classification", feature_names=X.columns, class_names=["Negative", "Positive"])
# Explain the true positive case
exp = explainer.explain_instance(ESD_X_true_positive.values[0], stacking.predict_proba, num_features=16, top_labels=1)

# Display the explanation
exp.show_in_notebook(show_table=True)


In [ ]:
# ----------------------------
# 8.3.2 True Negative Case in ESD
# ----------------------------
# Find the index of a true negative case in random forest, xgboost, and stacking
true_negative_idx = y_test[(y_test == 0) & (rf_pred == 0) & (gb_pred == 0) & (y_stacking_pred == 0)].index[0]

# Extract the features for the true negative case
ESD_X_true_negative = X_test.loc[[true_negative_idx]]
print("\nTrue Negative Case:")
print(ESD_X_true_negative)

In [ ]:
# Lime explainer for the true negative case via random forest
# Initialize the LimeTabularExplainer
explainer = LimeTabularExplainer(X_train.values, mode="classification", feature_names=X.columns, class_names=["Negative", "Positive"])
# Explain the true negative case
exp = explainer.explain_instance(ESD_X_true_negative.values[0], rf_best_model.predict_proba, num_features=16, top_labels=1)

# Display the explanation
exp.show_in_notebook(show_table=True)

In [ ]:
# Lime explainer for the true negative case via XGBoost
# Initialize the LimeTabularExplainer
explainer = LimeTabularExplainer(X_train.values, mode="classification", feature_names=X.columns, class_names=["Negative", "Positive"])
# Explain the true negative case
exp = explainer.explain_instance(ESD_X_true_negative.values[0], gb_best_model.predict_proba, num_features=16, top_labels=1)

# Display the explanation
exp.show_in_notebook(show_table=True)

In [ ]:
# Lime explainer for the true negative case via Stacking Classifier
# Initialize the LimeTabularExplainer
explainer = LimeTabularExplainer(X_train.values, mode="classification", feature_names=X.columns, class_names=["Negative", "Positive"])
# Explain the true negative case
exp = explainer.explain_instance(ESD_X_true_negative.values[0], stacking.predict_proba, num_features=16, top_labels=1)

# Display the explanation
exp.show_in_notebook(show_table=True)
